In [ ]:
#說明區
#本ipynb處理爬到的新聞內文


In [122]:
import requests
from goose3 import Goose
from goose3.text import StopWordsChinese
from goose3.text import StopWordsKorean
from goose3.text import StopWordsArabic
import pandas as pd

In [123]:
# goose抓內文
def main_text_goose(url) -> str:
    1
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
    content = ""
    try:
        url = requests.get(url).url # 取得原文連結(原先的url是被google news轉換過的)
        response = requests.get(url, headers=headers)
        if response.status_code<400:# 爬蟲沒被擋 
            # 偵測文章語言
            g = Goose() # 默認的method，若偵測不到文章語言，則會用這個method爬文章
            language = response.headers.get("Content-Language")
            if language != None:
                language = language.split('-')[0]
                g = Goose({'use_meta_language': False, 'target_language':language})
                # 中文、阿拉伯文、韓文等文章需要用到goose套件中的斷詞系統
                if language == 'zh':# 中文
                    g = Goose({'stopwords_class': StopWordsChinese})
                elif language == 'ar':# 阿拉伯文
                    g = Goose({'stopwords_class': StopWordsArabic})
                elif language == 'ko':# 韓文
                    g = Goose({'stopwords_class':StopWordsKorean})
                    
            # 爬取內文
            article = g.extract(raw_html=response.text)
            content = article.cleaned_text
            
            # 「文章有可能是中文，但先前沒偵測到文章語言」的額外處理
            if language == None and len(article.title) != 0 and len(article.cleaned_text) == 0:
                g = Goose({'stopwords_class': StopWordsChinese})
                article = g.extract(raw_html=response.text)
                content = article.cleaned_text 
            '''
            print("=====goose content=====")
            print(content)
            print("=================\n")'''
            # 檢查內文是否為空
            if len(content) > 0:
                return content   
            else:   
                print("goose抓不到內文")    
                print("文章網址: {}".format(url))
                return ""
        else:
            print("goose爬蟲被擋--Response Code:{}".format(response.status_code))
            print("文章網址: {}".format(url))
            return ""
    except Exception as e:
        print("can not get main text(goose)\n", e)
        return ""

In [172]:
# 將新聞的相關資料(標題、連結、內文等)寫入json檔
def writeNewsDataToJson(newsMedia = "",newsTitle = "",newsUrl = ""):
    # 爬取新聞內文
    text = ""
    text = main_text_goose(newsUrl)
    
    # 寫入json (url這一項要留著，用來判斷有沒有抓到重複的新聞)
    fileName = 'newsDatas.json'# json檔名稱
    newsData = {    # 要加入json檔的新聞
        'media':[newsMedia],  # 報導媒體
        'title':[newsTitle],  # 標題
        'text':[text],    # 內文
        'url':[newsUrl]   # 連結
    }
    df = pd.DataFrame()
    try:    
        df = pd.read_json(fileName)
    except FileNotFoundError:
        print('找不到json檔，將重新建立新的json檔')
        df = pd.DataFrame({'media': [], 'title': [],'text':[],'url':[]})
    except Exception:
        df = pd.DataFrame({'media': [], 'title': [],'text':[],'url':[]})
    if df.empty or (newsUrl not in df['url'].values):# 若json檔內沒有此篇新聞(以新聞連結判斷)，則加入此新聞
        df = pd.concat([df, pd.DataFrame(newsData)])
        df.to_json(fileName, orient='records', indent=4, force_ascii=False) 
    return
    